In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import openpyxl
import textwrap


#Die globalen Variablen dataframe wird erstellt, um sie später mit den Daten des Studiengangs zu befüllen
dataframe_input1 = pd.DataFrame()
dataframe_input2 = pd.DataFrame()


#Die Serien werden als leere Listen erstellt, um sie mit den Daten aus dem Studiengang zu befüllen und die Variablen in den dataframe zu ergänzen
serie_semester = []
serie_module = []
serie_status = []
serie_credits = []
serie_daten = []
serie_module2 = []
serie_ist_zeit = []

def serien_teil1_erzeugen(quelle):
    """
    Die Funktion serien_teil1_erzeugen ruft die Daten aus der Excel-Datei ab:

    Das Arbeitsblatt "Module" liefert die Daten zu Semestern, Modulen und Status.
    Die Daten werden in die zuvor erstellten leeren Listen gespeichert.

    Das Arbeitsblatt "Übersicht Zeitplan" liefert die Daten zu den Credits eines Moduls.
    Die Daten werden in die  Liste serie_credits gespeichert.
    """
    
    try:
        workbook = openpyxl.load_workbook(quelle) # Variable erzeugen um xlsx Datei einzulesen
        ws = workbook['Module']                   # Variable erzeugen, um das Arbeitsblatt Modul einzulesen
        rows = ws.iter_rows(min_row = 2,          # Schleifenvariable durch das Arbeitsblatt, Beginn bei Zeile 2
                            max_row = 33,         # Schleife endet bei Zeile 33
                            min_col = 1,          # Schleife beginnt in Spalte 1 (A)
                            max_col = 4,          # Schleife endet in Spalte 3 (C)
                            values_only = True)   # nur Werte auslesen, keine "Namen der Zellen"
        
        for a, b, c, d in rows:
            if isinstance(a, str) and isinstance(b, str) and isinstance(c, str) and isinstance(d, int):
                serie_semester.append(a)          # fügt die Werte der Spalte a in Liste serie_semester ein
                serie_module.append(b)            # fügt die Werte der Spalte a in Liste serie_module ein         
                serie_status.append(c)            # fügt die Werte der Spalte a in Liste serie_status ein
                serie_credits.append(d)           # # fügt die Werte der Spalte a in Liste serie_status ein
            else:
                raise ValueError(f"Einer der Werte im Tabllenblatt entspricht nicht dem erwarteten Datenformat! Korrigieren Sie den Wert!")

        
        ws = workbook['Ist-Zeit']                 # neues Arbeitsblatt mit Variable ws einlesen
        rows = ws.iter_rows(min_row = 2,          # ...
                            min_col = 1,          # ...        
                            max_col = 3,          # ...
                            values_only = True)   # ...
        for a, b, c in rows:
            if a == None or b == None or c == None:    # wenn eine Zelle keine Werte enthält,
                break                                  # dann endet die Schleife
            else:
                if not isinstance(a, datetime.datetime):
                    raise ValueError(f"Ein Wert im Tabllenblatt Ist-Zeit in Spalte A hat kein Datumsformat! Korrigieren Sie den Wert!")
                elif not isinstance(b, str):
                    raise ValueError(f"Ein Wert im Tabllenblatt Ist-Zeit in Spalte B ist keine Zeichenkette! Korrigieren Sie den Wert!")
                elif not isinstance(c, int):
                    raise ValueError(f"Ein Wert im Tabllenblatt Ist-Zeit in Spalte C ist nicht ganzzahlig! Korrigieren Sie den Wert!")
                else:
                    serie_daten.append(a)         # Datum wird in die Liste serie_daten geschrieben
                    serie_module2.append(b)       # Module wird in die Liste serie_module2 geschrieben
                    serie_ist_zeit.append(c)      # Ist-Zeit wird in die Liste serie_ist_zeit_geschrieben
            
    except FileNotFoundError:                     # falls Dateifehler
        print('Die Datei wurde nicht gefunden.')  # Hinweis auf falschen Pfad, oder falsche Datei


def dataframes_input_ergänzen():
    """
    Die Funktion dataframe_ergänzen fügt die erstellten Listen mit den Daten 
    aus der Excel-Datei in zwei verschiedene dataframes_input ein.
    """
    
    dataframe_input1['Semester'] = serie_semester
    dataframe_input1['Module'] = serie_module
    dataframe_input1['Status'] = serie_status
    dataframe_input1['Credits'] = serie_credits
    dataframe_input2['Daten'] = serie_daten
    dataframe_input2['Module'] = serie_module2
    dataframe_input2['Ist-Zeit'] = serie_ist_zeit

    dataframe_input1.sort_values(by='Semester', ascending = True, inplace = True)    # sortiert den Dataframe aufsteigend nach Semester

#--------------------------------------------------------------------------------------------------

class Studiengang:
    objekte = []                                  # Klassenliste, in der die objekte gespeichert werden
    dataframe = pd.DataFrame()                    # Klassendataframe, der als input für Diagramme dient
    
    def __init__(self,                            # Funktionsparameter Objekt
                 name,                            # Funktionsparameter Name
                 credits = 0,                     # Funktionsparameter Credits mit default-Wert
                 credits_erworben = 0,            # ...
                 start = None,                    # ...
                 ende = None,                     # ...
                 dauer = datetime.timedelta(),    # ...
                 ist_zeit = datetime.timedelta()):    # ...
        
        self.name = name                          # Objektattribut Name
        self.credits = credits                    # Objektattribut Credits
        self.credits_erworben = credits_erworben  # ...
        self.start = start                        # ...
        self.ende = ende                          # ...
        self.dauer = dauer                        # ...
        self.ist_zeit = ist_zeit                  # ...
        
    def soll_credits_einlesen(self):

        #Bildet die Summe aus Spalte "Credits" im ersten Dataframe input und übergibt den Wert an self.credits
        self.credits = int(dataframe_input1['Credits'].sum()) 

    def ist_credits_einlesen(self):

        #Filtert die Spalte "Status" und bildet die Summe aus Spalte "Credits" im ersten Dataframe input und übergibt den Wert an self.credits_erworben
        self.credits_erworben = int(dataframe_input1.loc[dataframe_input1['Status'] == 'bestanden', 'Credits'].sum())

    def start_erzeugen(self):
        self.start = datetime.date(2023, 9, 28)   # festes Datum für self.start vorgeben (Beginn des Studiums)

    def ende_erzeugen(self):
        self.ende = datetime.date(2027, 9, 27)    # festes Datum für self.start vorgeben (Ende des Studiums)

    def dauer_berechnen(self):
        self.dauer = self.ende - self.start       # Dauer ist die Zeit zwischen Ende und Start 

    def ist_zeit_berechnen(self):
        if datetime.date.today() > self.ende:     # wenn heutiger Tag später als Ende des Zeitabschnitts
            self.ist_zeit = self.dauer            # dann vergangene Zeit entspricht Dauer des Zeitabschnitts
        elif datetime.date.today() < self.start:  # wenn heutiger vor Studiumsbeginn liegt
            pass                                  # schreibe keinen Wert
        else:                                     # sonst
            self.ist_zeit = datetime.date.today() - self.start    # vergangene Zeit ist Zeitraum von Start des Zeitabschnitts bis heute
        

    def dataframe_output_erzeugen(self):
        """ Die Funktion dataframe_output_erzeugen erzeugt einen Dataframe, der alle Daten der Objekte in einer Tabelle
        zusammenfasst und zusätzlich als Grundlage für die Erstellung der Diagramme dient, um Code zu sparen"""
        
        neue_reihe = pd.DataFrame({'Credits' : [self.credits],                         # Dataframe mit Spalte "Credits"
                                   'Credits erworben' : [self.credits_erworben],       # und Spalte "Credits erworben
                                   'Start' : [self.start],                             # und Startdatum
                                   'Ende' : [self.ende],                               # und Endedatum
                                   'Dauer' : [self.dauer],                             # und Dauer
                                   'Ist-Zeit' : [self.ist_zeit]}, index = [self.name]) # und Ist-Zeit
        self.__class__.dataframe = pd.concat([self.__class__.dataframe, neue_reihe])   # leerer Dataframe wird mit "neue Reihe" ergänzt



class Semester(Studiengang):                      # erbt alle Attribute von Klasse Studiengang, keine eigene init-Funktion
    objekte = []                                  # muss neu definiert werden, sonst erbt die Liste auch Inhalte der Studiengangklasse
    dataframe = pd.DataFrame()                    # muss neu definiert werden, sonst erbt der Dataframe auch Inhalte der Studiengangklasse
    
    def soll_credits_einlesen(self):              # Funktion aus Studiengangklasse wird überschrieben

        #w1 neuer Dataframe erzeugt, der die Spalte Semester für alle Zeilen mit gleichem Namen aufsummiert (z.B. Semester1 = 25)
        dataframe_input1_neu = dataframe_input1.groupby([f'{self.__class__.__name__}']).sum()

        #w2 neuer Dataframe durchsucht die Zeilen nach dem Attribut Namen und übergibt den Wert in Spalte "Credits" an self.credits
        self.credits += int(dataframe_input1_neu.loc[self.name].at['Credits'])

    def ist_credits_einlesen(self):               # Funktion aus Studiengangklasse wird überschrieben
        try:
            #w1 neuer Dataframe erzeugt, der die Spalte Semester für alle Zeilen mit gleichem Namen aufsummiert und nach "Status" bestanden filtert 
            dataframe_input1_neu = dataframe_input1[dataframe_input1['Status'] == 'bestanden'].groupby([f'{self.__class__.__name__}']).sum()

            #w2 neuer Dataframe durchsucht die Zeilen nach dem Attribut Namen und übergibt den Wert in der Spalte "Credits" an self.credits
            self.credits_erworben += int(dataframe_input1_neu.loc[self.name].at['Credits'])
            
        except KeyError:                          # falls Zeile in dataframe_input nicht bestanden, dann KeyError
            self.credits_erworben = 0             # dann gib den Wert 0 an self.credits



class Module(Semester):                           # erbt alle Attribute von Klasse Semester
    objekte = []                                  # ...
    dataframe = pd.DataFrame()                    # ...

    def __init__(self, name, status = None):      # zusätzliches Attribut Status eingefügt, wird für Diagramm rechts unten verwendet
        super().__init__(name)                    # ruft init Funktion von Klasse Semester auf
        self.status = status                      # Attribut Status = Funktionsname Status

    def ist_zeit_berechnen(self):                 # Funktion aus Studiengangklasse wird überschrieben
        try:
            #w1 neuer Dataframe erzeugt, der die Spalte Semester für alle Zeilen mit gleichem Namen aufsummiert und nach "Status" bestanden filtert
            dataframe_input1_neu = dataframe_input1[dataframe_input1['Status'] == 'bestanden'].groupby([f'{self.__class__.__name__}']).sum()
            
            #w2 neuer Dataframe durchsucht die Zeilen nach dem Attribut Namen und übergibt den Wert in Spalte "Credits"*30
            # es wird mit 30 multipliziert, da 5 Credits = 150 Stunden, für bestandene Module wird Modul-Ist-Zeit der Soll-Zeit gleichgesetzt
            x = int(dataframe_input1_neu.loc[self.name].at['Credits']*30)    # int type, da dataframes numpy.int64 nicht kompatibel mit datetime
            self.ist_zeit = datetime.timedelta(hours = x)
        
        except KeyError:                          # falls Zeile in dataframe_input nicht bestanden, dann KeyError
            try:
                # neuer Dataframe erzeugt, der nach "Status" angemeldet filtert
                dataframe_input1_neu = dataframe_input1[dataframe_input1['Status'] == 'angemeldet']

                # neuer Dataframe aus !dataframe_input2! erzeugt, der nach Spalten "Module" und "Ist-Zeit" filtert
                #Filter ist wegen sum-Funktion zwei Codezeilen später notwendig (Datumstypen können nicht aufsummiert werden)
                dataframe_input2_neu = dataframe_input2[['Module','Ist-Zeit']]

                # an den neuen Dataframe1 wird Spalte Ist-Zeit angefügt
                dataframe_input1_neu = pd.merge(dataframe_input1_neu, dataframe_input2_neu, how = 'left', on = 'Module')

                # alle Ist-Zeiten mit den gleichen Modulnamen werden aufsummiert (wie SUMMEWENN Funktion in Microsoft Excel)
                dataframe_input1_neu = dataframe_input1_neu.groupby(['Module']).sum()
                
                # neuer Dataframe durchsucht die Zeilen nach dem Attribut Namen und übergibt den Wert in der Spalte "Ist-Zeit"
                x = int(dataframe_input1_neu.loc[self.name].at['Ist-Zeit'])
                self.ist_zeit = datetime.timedelta(minutes = x)
            
            except KeyError:                      # falls Zeile in dataframe_input auch nicht angemeldet, dann KeyError
                self.ist_zeit = datetime.timedelta()    # dann Ist-Zeit = 0

    def status_einlesen(self):

        # erzeugt neuen Dataframe mit der Spalte "Status", die Werte dafür stammen aus dataframe_input1 Spalte "Status"
        # und verwendet als Index den Namen des Moduls
        # die Funktion tolist ist notwendig, um die 'Spalten' des datamframes_input1 in Listen umzuwandeln und an den neuen Dataframe zu übergeben
        dataframe_input1_neu = pd.DataFrame({'Status' : dataframe_input1['Status'].tolist()}, index = dataframe_input1['Module'].tolist())
        # neuer Dataframe durchsucht die Zeilen nach dem Attribut Namen und übergibt den Wert in der Spalte "Status"
        self.status = dataframe_input1_neu.loc[self.name].at['Status']
        
    def dauer_berechnen(self):                    # Funktion aus Studiengangklasse wird überschrieben
        self.dauer = datetime.timedelta(hours = self.credits*30)    # Dauer entspricht Credits * 30h



class Wochen(Semester):                           # erbt alle Attribute von Klasse Semester, nicht von Modul, da kein Attribut 'Status' notwendig
    objekte = []                                  # ...
    dataframe = pd.DataFrame()                    # ...
    

    def ist_zeit_berechnen(self):                 # Funktion aus Studiengangklasse wird überschrieben

        # erzeugt neuen Dataframe und filtert die Daten nach später oder gleich self.start und früher oder gleich self.ende
        # die Methode dt.date (datetime.date) ist notwendig um das Datum aus dem Dataframe mit dem Datum aus Attribut start vergleichen zu können
        dataframe_input2_neu = dataframe_input2[(dataframe_input2['Daten'].dt.date >= self.start) & (dataframe_input2['Daten'].dt.date <= self.ende)]

        # summiert die Minuten für die gefilterten Kriterien auf und übergibt sie als Minuten an self.ist_zeit
        self.ist_zeit = datetime.timedelta(minutes = int(dataframe_input2_neu['Ist-Zeit'].sum()))

#--------------------------------------------------------------------------------------------------

def objekte_erzeugen():
    """ Die Funktion objekte_erzeugen erzeugt die Objekte der einzelnen Klassen"""
    
    studiengang = Studiengang("Angewandte Künstliche Intelligenz")
    Studiengang.objekte.append(studiengang)       # fügt das erzeugte Objekt in die Klassenliste ein
    doppelter_wert = 0                            # notwendig um doppelte Semesterwerte auszuschließen
    for element in dataframe_input1['Semester']:  #w3 durchläuft die Spalte "Semester"
        if doppelter_wert != element:             # wenn objekt noch nicht vorhanden, dann
            semester = Semester(element)
            Semester.objekte.append(semester)     # fügt das erzeugte Objekt in die Klassenliste ein
            doppelter_wert = element              # setzt die Variable auf das zuletzt eingefügte Objekt
        else:
            pass
    for element in dataframe_input1['Module']:    #w3 ...
        if doppelter_wert != element:             # ...
            module = Module(element)
            Module.objekte.append(module)         # ...
            doppelter_wert = element              # ...
        else:
            pass
    woche_soll = Wochen('Soll pro Woche')         # das Objekt soll die Sollzeit pro Woche beinhalten
    woche_ist_durchschnitt = Wochen('durchschnittliches Ist pro Woche') # das Objekt soll die durchschnittliche Ist-Zeit pro Woche beinhalten
    Wochen.objekte.append(woche_soll)             # ...
    Wochen.objekte.append(woche_ist_durchschnitt) # ...
    for i in range(1,5):
        woche = Wochen('x{i}')                    # erzeugt vier Objekte, die die Daten der letzten vier Wochen beinhalten
        woche.start = datetime.date.today() - datetime.timedelta(days = 7*(i)) # Startzeitpunkt der Wochenobjekte
        woche.ende = woche.start + datetime.timedelta(days = 6) # Endzeitpunkt der Wochenobjekte
        woche.name = (f'Woche vom {woche.start} bis {woche.ende}')
        Wochen.objekte.append(woche)              # ...
    

def attribut_start_erzeugen():                    # erzeugt die Startdaten für die Semester, 
    for i in range(len(Semester.objekte)):        # funktioniert nur richtig, wenn die Semester bereits sortiert in der Excel-Datei stehen
        Semester.objekte[i].start_erzeugen()      # schreibt in alle Semesterobjekte der Liste das Startdatum des Studiums
        if i > 0:                                 # für alle Semester, die nicht das erste Semester sind
            if Semester.objekte[i-1].start.month > 6:     # wenn vorheriges Semester in Monat > 6 startet

                # dann ziehe vom Startmonat des vorherigen Semesters 6 ab und addiere für das Jahr 1 = Startmonat aktuelles Semester
                Semester.objekte[i].start = Semester.objekte[i-1].start.replace(year = Semester.objekte[i-1].start.year + 1,
                                                                                month = Semester.objekte[i-1].start.month - 6)
            else:
                # sonst: addiere zum Startmonat des vorherigen Semesters 6 = Startmonat aktuelles Semester
                Semester.objekte[i].start = Semester.objekte[i-1].start.replace(month = Semester.objekte[i-1].start.month + 6)
                
        else:                                                  
            pass                                  # für das erste Semester wird der Startwert nicht verändert

def attribut_ende_erzeugen():                     # erzeugt die Enddaten für die Semester
    for i in range(len(Semester.objekte)):        # funktioniert nur richtig, wenn die Semester bereits sortiert in der Excel-Datei stehen
        Semester.objekte[i].ende_erzeugen()       # schreibt in alle Semesterobjekte der Liste das Startdatum des Studiums
        if i < 7:                                 # für alle Semester, die nicht das letzte Semester sind
                                                  # Das Semesterende liegt ein Tag vor dem Start des nachfolgenden Semester
            Semester.objekte[i].ende = Semester.objekte[i+1].start.replace(day = Semester.objekte[i+1].start.day - 1)
        else:
            pass                                  # für das letzte Semester wird der Endwert nicht verändert


#--------------------------------------------------------------------------------------------------

def main():

    # Funktionsaufruf um Listen aus Excel-Datei zu erzeugen
    serien_teil1_erzeugen(input("Geben Sie den Dateipfad der einzulesenden Excel-Datei an."))
    dataframes_input_ergänzen()                   # Funktionsaufruf dataframes aus Listen erzeugen
    objekte_erzeugen()                            # Funktionsaufruf objekte erzeugen
    Studiengang.objekte[0].soll_credits_einlesen()    #Funktionsaufruf für Klasse Studiengang
    for objekt in Semester.objekte:               # für alle Semesterobjekte
        objekt.soll_credits_einlesen()            # Funktionsaufruf soll_credits_einlesen
        objekt.ist_credits_einlesen()             # ...
    for objekt in Module.objekte:                 # für alle Moduleobjekte
        objekt.soll_credits_einlesen()            # ...
        objekt.ist_credits_einlesen()             # ...
        objekt.ist_zeit_berechnen()               # ...
        objekt.status_einlesen()                  # ...
    Studiengang.objekte[0].ist_credits_einlesen()    # Funktionsaufruf für Klasse Studiengang
    Studiengang.objekte[0].start_erzeugen()       # ...
    Studiengang.objekte[0].ende_erzeugen()        # ...
    attribut_start_erzeugen()                     # Funktionsaufruf um Startwerte der Semester zu generieren
    attribut_ende_erzeugen()                      # Funktionsaufruf um Startwerte der Semester zu generieren
    Studiengang.objekte[0].dauer_berechnen()      # Funktionsaufruf macht erst Sinn nach Funktionsaufruf Start und Ende
    Studiengang.objekte[0].ist_zeit_berechnen()   # Funktionsaufruf macht erst Sinn nach Funktionsaufruf Start
    for objekt in Semester.objekte:               # ...
        objekt.dauer_berechnen()                  # Funktionsaufruf macht erst Sinn nach Funktionsaufruf Start und Ende
        objekt.ist_zeit_berechnen()               # Funktionsaufruf macht erst Sinn nach Funktionsaufruf Start
    for objekt in Module.objekte:                 # ...
        objekt.dauer_berechnen()                  # Funktionsaufruf macht erst Sinn nach Funktionsaufruf Credits_einlesen
    for objekt in Wochen.objekte:
        objekt.ist_zeit_berechnen()
    Studiengang.objekte[0].dataframe_output_erzeugen()    # Funktionsaufruf dataframe_output für Studiengangobjekte
    for objekt in Semester.objekte:
        objekt.dataframe_output_erzeugen()        # Funktionsaufruf dataframe_output für Studiengangobjekte
    for objekt in Module.objekte:
        objekt.dataframe_output_erzeugen()        # ...
    status_liste = []
    for i in range(len(Module.objekte)):
        status_liste.append(Module.objekte[i].status)
    Module.dataframe['Status'] = status_liste     # Spalte 'Status' der Module wird an bestehenden Output_Dataframe angefügt

    # legt den Wert der wöchentlichen Soll_Arbeitszeit fest
    Wochen.objekte[0].ist_zeit = datetime.timedelta(hours = Studiengang.objekte[0].credits*30 / (Studiengang.objekte[0].dauer.total_seconds()/60/60/24/7))

    # legt den Wert der durchschnittlichen Ist-Arbeitszeit fest, kann erst nach Erzeugen des Dataframes_output für Module berechnet werden
    Wochen.objekte[1].ist_zeit = datetime.timedelta(hours = Module.dataframe['Ist-Zeit'].sum().total_seconds()/60/60 / (Studiengang.objekte[0].ist_zeit.total_seconds()/60/60/24/7))
    for objekt in Wochen.objekte:
        objekt.dataframe_output_erzeugen()        # Funktionsaufruf dataframe_output für Studiengangobjekte
     

#--------------------------------------------------------------------------------------------------

    # Bild mit 4 Diagrammen und Breite 14,8 und Höhe 10,5
    fig, axes = plt.subplots(2, 2, figsize=(14.8, 10.5), layout = "constrained")


    
    # Diagramm oben rechts
    
    colors = []                                   # leere Liste für Farbvariablen

    # erste Datenliste für Kreisdiagramm oben rechts, erworbene Credits und verbleibende Credits (äußerer Ring)
    sizes1 = [Studiengang.objekte[0].credits_erworben, Studiengang.objekte[0].credits - Studiengang.objekte[0].credits_erworben]
    
    sizes2 = []                                   # leere Liste für Daten Kreisdiagramm (innerer Ring)
    labels = []                                   # leere Liste für Bezeichnung der Kreisdiagrammfragmente
    for i in range(len(Semester.objekte)):        
        sizes2.append(Semester.objekte[i].credits_erworben)    # fügt Wert der erworbenen Credits ein
        colors.append('lightgreen')                            # fügt in die Farbliste hellgrün ein (für erworbene Credits)
        if Semester.objekte[i].credits_erworben == 0:          # wenn im Semester noch keine Credits erworben wurden
            label = ''                                         # dann Name des Kreisdiagrammfragments soll nicht gezeigt werden
        else:
            label = Semester.objekte[i].name                   # sonst Kreisdiagrammfragment trägt den Semesternamen
        labels.append(label)

        # danach werden die noch verbleibenden Credits des Semester in die Liste sizes 2 eingefügt
        sizes2.append(Semester.objekte[i].credits - Semester.objekte[i].credits_erworben)
        if Semester.objekte[i].credits - Semester.objekte[i].credits_erworben == 0:    # wenn im Semester keine Credits verbleiben
            label = ''                                         # Name des Kreisdiagrammfragments soll nicht gezeigt werden
        else:
            label = Semester.objekte[i].name                   # sonst Kreisdiagrammfragment trägt den Semesternamen
        labels.append(label)
        colors.append('0.7')                                   # fügt Farbe "hellgrau" in die FarbListe ein (für verbleibende Credits)
    
    # Diagramm oben rechts ist Kreisdiagramm
    axes[0,1].pie(sizes1,                         # Daten für äußeren Datenring
                  radius=1,                       # Radius = 1
                  colors=['g','0.3'],             # Farben sind grün und dunkelgrau
                  startangle = 90,                # Start des Kreisdiagramm ist oben ("12 Uhr")
                  counterclock = False,           # gegen den Uhrzeigersinn
                  wedgeprops=dict(width=0.25, edgecolor='black'))    # Breite der Kreisdiagrammfragmente =0,25, Randfarbe schwarz

    # Diagramm oben rechts ist Kreisdiagramm
    axes[0,1].pie(sizes2,                         # Daten für inneren Datenring
                  radius=1-0.25,                  # schließt an den äußeren Ring an mit Radius 0,75
                  colors=colors,                  # Farben für erworben und verbleibend je Semester
                  labels = labels,                # Semesternamen
                  labeldistance = 1.6,            # Abstand der Kreisdiagrammfragentbezeichnungen vom Kreisdiagramm
                  startangle = 90,                # ...
                  counterclock = False,           # ...
                  wedgeprops=dict(width=0.25, edgecolor='black'))    # ...

    # Titel des Kreisdiagramms oben rechts, Schriftgröße 20, Abstand von Diagramm 20, Überschriftsfrabe
    axes[0,1].set_title('Credits - erworben (grün) und verbleibend (grau)',fontsize = 20, pad = 20, color = 'dimgrey')


    # Diagramm oben links
    
    #w4 Kreisdiagramm oben links analog zu Kreisdiagramm oben rechts erstellt
    colors = []
    sizes1 = [Studiengang.objekte[0].ist_zeit.total_seconds(), (Studiengang.objekte[0].dauer - Studiengang.objekte[0].ist_zeit).total_seconds()]
    sizes2 = []
    labels = []
    for i in range(len(Semester.objekte)):
        sizes2.append(Semester.objekte[i].ist_zeit.total_seconds())
        colors.append('lightgreen')
        if Semester.objekte[i].ist_zeit == datetime.timedelta():
            label = ''
        else:
            label = Semester.objekte[i].name
        labels.append(label)
        sizes2.append((Semester.objekte[i].dauer - Semester.objekte[i].ist_zeit).total_seconds())
        if Semester.objekte[i].dauer - Semester.objekte[i].ist_zeit == datetime.timedelta():
            label = ''
        else:
            label = Semester.objekte[i].name
        labels.append(label)
        colors.append('0.7')
    
    # ...
    axes[0,0].pie(sizes1, radius=1.0, colors=['g','0.3'], startangle = 90, counterclock = False, wedgeprops=dict(width=0.25, edgecolor='black'))

    # ...
    axes[0,0].pie(sizes2, radius=1-0.25, colors=colors, labels = labels, labeldistance = 1.6, startangle = 90, counterclock = False, wedgeprops=dict(width=0.25, edgecolor='black'))

    # ...
    axes[0,0].set_title('Studienzeit in Tagen - vergangen (grün)\n und verbleibend (grau)',fontsize = 20, pad = 20, color = 'dimgrey')



    # Diagramm unten rechts
    
    # Gestapeltes Säulendiagramm unten rechts mit Werten von Ist-Zeiten für angemldete Module und verbleibende Zeiten für angemeldete Module
    liste1 = Module.dataframe[Module.dataframe['Status'] == 'angemeldet']['Ist-Zeit'].tolist()
    liste2 = (Module.dataframe[Module.dataframe['Status'] == 'angemeldet']['Dauer'] - 
              Module.dataframe[Module.dataframe['Status'] == 'angemeldet']['Ist-Zeit']).tolist()
    
    sizes1 = []                                   # Datenreihe für untere Säulen   
    sizes2 = []                                   # Datenreihe für obere Säulen
    for i in liste1:
        i = i.total_seconds()/(60*60)             # wandelt die datetimeInstanzen in einen float-Typ um
        sizes1.append(i)                          # fügt die Ist-Zeiten in die unteren Säulen ein
    for i in liste2:
        i = i.total_seconds()/(60*60)             # ...
        i = max(i, 0)                             # ... wenn verbleibend kleiner als 0, dann 0
        sizes2.append(i)                          # fügt die verbleibenden Zeiten in die oberen Säulen ein

    # Variable für X-Achsen Bezeichnung (Index des Dataframes_output Module)  
    module = Module.dataframe[Module.dataframe['Status'] == 'angemeldet'].index.tolist()
    
    # fill Methode aus Wrap Modul wird benötigt, um Text passend zu arrangieren, Textweite maximal 10, lange Wörter werden nicht zerteilt
    module_wrapped = [textwrap.fill(label,width = 10, break_long_words = False) for label in module]

    # Zeitenvariable um numpy arrays aus den Listenwerte zu erstellen (numpy arrays können elementweise addiert werden, Listen nicht)
    zeiten = {'erledigt' : np.array(sizes1), 'verbleibend' : np.array(sizes2)}
    
    # für die Anzahl Module mit dem "Status" angemeldet wird ein numpy Array mit gleich vielen Nullen erstellt (Boden des Säulendiagramms)
    bottom = np.zeros(int(Module.dataframe[Module.dataframe['Status'] == 'angemeldet']['Ist-Zeit'].count()))

    # untere Säulen werden alle grün, es sei denn die verbleibende Dauer im Modul ist < 0, dann wird die Säule orange/chocolate
    colors = []
    for i in range(len(sizes1)):
        if liste2[i] < datetime.timedelta():
            colors.append('chocolate')
        else:
            colors.append('g')

    # dictionary mit Säulenwerten erstellen
    for key, values in zeiten.items():
        
        # unten rechts - Säulendiagramm für untere Säulen
        axes[1,1].bar(module_wrapped,             # Modulnamen formatiert als Säulenbeschriftung
                      values,                     # Modul Ist-Zeiten als Säulenwerte
                      width = 0.5,                # maximale Säulenbreite
                      bottom = bottom,            # ...
                      color = colors,             # ...
                      edgecolor = 'black',        # ...
                      zorder = 4)                 # Säulen liegen im Vordergrund (vor den Gitternetzlinien mit default = 3)
        
        bottom += values                          # obere Säulen setzen an oberer Kante der unteren Säulen fort
        colors.clear()                            # Farbliste leeren, um Farben für obere Säulen einzufügen
        
        # obere Säulen werden alle dunkelgrau
        for i in range(int(Module.dataframe[Module.dataframe['Status'] == 'angemeldet']['Ist-Zeit'].count())):
            colors.append('0.3')

    # Diagrammrahmen für Diagramm unten rechts wird ausgeblendet
    for spine in axes[1,1].spines.values():
        spine.set_visible(False)
    
    # Gitternetzlinien für Hauptachsenwerte an der Y-Achse in hellgrau einfügen
    axes[1,1].grid(which='major', axis='y', color = 'lightgray')
    
    # X-Achsenbeschriftung mit 0° Drehung und kleiner Buchstabengröße (Argument für Parameter Drehung ist nicht notwendig)
    axes[1,1].tick_params(axis='x', rotation=0, labelsize = 'small')
    
    # Y-Achsenbeschriftung mit Schriftgröße 16
    axes[1,1].set_ylabel('Stunden', fontsize = 16)
    
    # ...
    axes[1,1].set_title('\nArbeitsaufwand aktuelle Module -\n erledigt (grün) und verbleibend (grau)', fontsize = 20, pad = 20, color = 'dimgrey')



    # Diagramm unten links (Säulendiagrammm)
    
    
    bottom = np.zeros(6)                          # Anzahl der Säulen im Diagramm = 6
    colors = []                                   # ...
    wochen = Wochen.dataframe.index.tolist()      # Säulenbeschriftung

    # Säulenbeschriftung formatiert 
    wochen_wrapped = [textwrap.fill(label, width = 10, break_long_words = False) for label in wochen]
    
    
    liste1 = Wochen.dataframe['Ist-Zeit'].tolist()    # Zeiten für alle Wochen in Liste speichern
    sizes1 = []
    for i in liste1:
        i = i.total_seconds()/(60*60)                 # Zeiten in float typ unwandeln
        sizes1.append(i)
        
        # wenn Zeit des Wochenobjekts größer als Soll-Zeit pro Woche, dann Farbe grün
        if i > Wochen.objekte[0].ist_zeit.total_seconds()/(60*60):
            color = 'g'
        
        # wenn Zeit des Wochenobjekts zwischen 75% und 100% der Sollzeit, dann farbe Gold
        elif i < (Wochen.objekte[0].ist_zeit.total_seconds()/(60*60)) and i >= (Wochen.objekte[0].ist_zeit.total_seconds()/(60*60))*0.75:
            color = 'gold'
        
        # wenn Zeit des Wochenobjekts zwischen 50% und 75% der Sollzeit, dann farbe Chocolate/Orange 
        elif i < (Wochen.objekte[0].ist_zeit.total_seconds()/(60*60))*0.75 and i >=(Wochen.objekte[0].ist_zeit.total_seconds()/(60*60))*0.5:
            color = 'chocolate'
        
        # wenn Zeit des Wochenobjekts kleiner als 50% der Sollzeit, dann farbe Dunkelrot
        elif i < (Wochen.objekte[0].ist_zeit.total_seconds()/(60*60))*0.5:
            color = 'darkred'
        
        # sonst (Wenn Zeit des Wochenobjekts = der Sollzeit; trifft nur auf SollZeit Wochenobjekt zu), dann Farbe dunkelgrau
        else:
            color = '0.3'
        colors.append(color)

    # ...
    for spine in axes[1,0].spines.values():
        spine.set_visible(False)

    # ...
    axes[1,0].grid(which='major', axis='y', color = 'lightgray')
    
    # ...
    axes[1,0].bar(wochen_wrapped, sizes1, width = 0.5, bottom = bottom, color = colors, edgecolor = 'black', zorder = 4)
    
    # ...
    axes[1,0].set_ylabel('Stunden', fontsize = 16)
    
    # ...
    axes[1,0].set_title('\nArbeitsaufwand Gesamt pro Woche',fontsize = 20, pad = 20, color = 'dimgrey')

    # Titel mit Namen des Studiengangs und Datum einfügen
    fig.suptitle(f'Studiengang: {Studiengang.objekte[0].name} (Datum: {datetime.date.today()})\n', size = 20)
    
if __name__ == "__main__":
    main()